In [ ]:
import pandas as pd
import numpy as np
import json
import re
import requests
import string
import matplotlib.pyplot as plt

import spacy
from textblob import TextBlob
tokens = spacy.load("en_core_web_sm")
import datasets
from datasets import load_dataset
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import norm
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.metrics import edit_distance
nltk.download('wordnet')
nltk.download("stopwords")
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#quotes_path = '/content/drive/MyDrive/NLP Project/quotes.json'
#phrases_path = '/content/drive/MyDrive/NLP Project/English_phrases_and_sayings.csv'
movies_path = '/content/drive/MyDrive/NLP Project/tmdb_5000_movies.csv'
#music_path = '/content/drive/MyDrive/NLP Project/tcc_ceds_music.csv'
words_path = '/content/drive/MyDrive/NLP Project/word-meaning-examples.csv'
news_path = '/content/drive/MyDrive/NLP Project/cnn_data.json'
#formal_idioms_path = '/content/drive/MyDrive/NLP Project/Formal_idioms.txt'
#formal_idioms_meaning_path = '/content/drive/MyDrive/NLP Project/Formal_idioms_meanings.txt'
#static_idioms_path = '/content/drive/MyDrive/NLP Project/Static_idioms.txt'
#static_idioms_meaning_path = '/content/drive/MyDrive/NLP Project/Static_idioms_meanings.txt'
animals_path = '/content/drive/MyDrive/NLP Project/animals.json'
cars_path = '/content/drive/MyDrive/NLP Project/cars.json'
city_country_path = '/content/drive/MyDrive/NLP Project/city-country.json'
food_path = '/content/drive/MyDrive/NLP Project/food.json'
literature_path = '/content/drive/MyDrive/NLP Project/literature.json'
people_path = '/content/drive/MyDrive/NLP Project/people.json'
sports_path = '/content/drive/MyDrive/NLP Project/sports.json'
things_path = '/content/drive/MyDrive/NLP Project/things.json'
media_path = '/content/drive/MyDrive/NLP Project/tv.json'
web_path = '/content/drive/MyDrive/NLP Project/web.json'
glove_path = '/content/drive/MyDrive/NLP Project/glove.6B.100d.txt'

In [ ]:
dictionary_link = "marksverdhei/wordnet-definitions-en-2021"

dictionary_train = load_dataset(dictionary_link, split="train")
dictionary_val = load_dataset(dictionary_link, split="validation")
dictionary_test = load_dataset(dictionary_link, split="test")

word1 = dictionary_train['Word']
example1 = dictionary_train['Example']
definition1 = dictionary_train['Definition']
word2 = dictionary_val['Word']
example2 = dictionary_val['Example']
definition2 = dictionary_val['Definition']
word3 = dictionary_test['Word']
example3 = dictionary_test['Example']
definition3 = dictionary_test['Definition']

dictionary_train = pd.DataFrame({"Word": word1, "Example": example1, "Definition": definition1})
dictionary_val = pd.DataFrame({"Word": word2, "Example": example2, "Definition": definition2})
dictionary_test = pd.DataFrame({"Word": word3, "Example": example3, "Definition": definition3})
dictionary_data = pd.concat([dictionary_train, dictionary_val, dictionary_test], ignore_index=True)

rdictionary_data = pd.DataFrame(columns=['Target', 'Clues'])

for index, row in dictionary_data.iterrows():
    target = f"{row['Word']}"
    clues = f"{row['Example']} {row['Definition']}"

    text_blob_clues = TextBlob(clues)
    noun_phrases_clues = ' '.join(text_blob_clues.noun_phrases)

    rdictionary_data = pd.concat([rdictionary_data, pd.DataFrame({'Target': target, 'Clues': [noun_phrases_clues]})], ignore_index=True)

rdictionary_data = rdictionary_data[rdictionary_data['Target'].apply(lambda x: len(x) > 0)]
rdictionary_data = rdictionary_data[rdictionary_data['Clues'].apply(lambda x: len(x) > 0)]
rdictionary_data.sample(10)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


,Target,Clues
32744,choric,greek tragedy greek chorus
36735,gluttonously,man eats gluttonous manner
25710,touch,pressure receptors
14722,clerical,clerical job office work
26367,paradigm,psychoanalytic paradigm particular discipline
6392,technical,technical report
1472,flyblown,flyblown reputation
40404,inflame,repetitive motion
34650,here and there,various places location pronoun
38766,flick,sharp contact


In [ ]:
import spacy
sub_model = spacy.load("en_core_web_sm")

news_data = pd.read_json(news_path)
news_data = news_data[["title", "short_description"]]
rnews_data = pd.DataFrame(columns=['Target', 'Clues'])

def extract_subject(sentence):
    doc = sub_model(sentence)
    for token in doc:
        if "subj" in token.dep_:
            return token.text
    return None

def process_dataset(dataset):
    subjects = []
    for sentence in dataset['title']:
        subject = extract_subject(sentence)
        subjects.append(subject)
    return subjects

news_data['title'] = process_dataset(news_data)

for index, row in news_data.iterrows():
    target = f"{row['title']}"
    clues = f"{row['short_description']}"

    text_blob_target = TextBlob(target)
    text_blob_clues = TextBlob(clues)
    noun_phrases_target = ', '.join(text_blob_target.noun_phrases)
    noun_phrases_clues = ', '.join(text_blob_clues.noun_phrases)

    rnews_data = pd.concat([rnews_data, pd.DataFrame({'Target': [noun_phrases_target], 'Clues': [noun_phrases_clues]})], ignore_index=True)

rnews_data = rnews_data[rnews_data['Target'].apply(lambda x: len(x) > 0)]
rnews_data = rnews_data[rnews_data['Clues'].apply(lambda x: len(x) > 0)]
rnews_data.sample(10)

,Target,Clues
3226,ferrer,"andy roddick, u.s., open champion, juan martin..."
631,fia,"motorsport, 's ruling body, team name changes"
1385,mickelson,"four-time, major winner, phil mickelson, week ..."
2831,corinthians,"brazilian, corinthians, america, aggregate def..."
3168,shin,"korea, jiyai shin, eight-under par round, cour..."
2404,murdoch,"media, rupert murdoch, fit person, major inter..."
2157,poland,"poland, vast natural resource"
1608,ranieri,"claudio ranieri, inter milan, dismal run, euro..."
3276,mcilroy,"golf 's, enthusiastic crowd, china, rory mcilr..."
3382,liverpool,"english, premier league giants, liverpool, spi..."


In [ ]:
def generate_df(json_file_path):
    data = pd.read_json(json_file_path, orient='index')
    data.reset_index(inplace=True)
    numerical_columns = [col for col in data.columns if str(col).isdigit()]
    data['num_cols'] = data[numerical_columns].apply(lambda row: ', '.join([str(val) for val in row if pd.notnull(val)]), axis=1)
    data = data.drop(columns=numerical_columns)
    data.columns = ['Target', 'Clues']
    return data

animals_data = generate_df(animals_path)
cars_data = generate_df(cars_path)
city_country_data = generate_df(city_country_path)
food_data = generate_df(food_path)
literature_data = generate_df(literature_path)
people_data = generate_df(people_path)
things_data = generate_df(things_path)
media_data = generate_df(media_path)
web_data = generate_df(web_path)

word_data = pd.concat([animals_data, cars_data, city_country_data, food_data, literature_data, people_data, things_data, media_data, web_data])
word_data.sample(10)

,Target,Clues
66,Rover,"England, Tata, small cars, BWM"
8,Apocalypse Now,"Francis Ford Coppola, Vietnam, helicopters, U...."
20,Der Blade Runner,"Action, Sci Fi, Harrison Ford, original"
199,Rachel Weisz,"actress, The Mummy, The Constant Gardener, The..."
67,The Complete Fairy Tales,Hans Christian Andersen
327,Port-aux-Français (French Southern Territories),Africa
204,Slovakia,"Europe, Eastern Europe, Bratislava"
21,Arnold Schwarzenegger,"actor, Hollywood, USA, Terminator, I'll be back"
24,Jacques the Fatalist,Denis Diderot
21,Chihuahua,"Paris Hilton, dog, small, foot horn"


In [ ]:
movies_data = pd.read_csv(movies_path)
movies_data = movies_data[["title", "overview"]]
rmovies_data = pd.DataFrame(columns=['Target', 'Clues'])

for index, row in movies_data.iterrows():
    target = f"{row['title']}"
    clues = f"{row['overview']}"

    text_blob_target = TextBlob(target)
    text_blob_clues = TextBlob(clues)
    noun_phrases_target = ' '.join(text_blob_target.noun_phrases)
    noun_phrases_clues = ', '.join(text_blob_clues.noun_phrases)

    rmovies_data = pd.concat([rmovies_data, pd.DataFrame({'Target': [noun_phrases_target], 'Clues': [noun_phrases_clues]})], ignore_index=True)

rmovies_data = rmovies_data[rmovies_data['Target'].apply(lambda x: len(x) > 0)]
rmovies_data = rmovies_data[rmovies_data['Clues'].apply(lambda x: len(x) > 0)]
rmovies_data.sample(10)

,Target,Clues
868,elizabethtown,"drew baylor, shoe company, drew, new lease, fa..."
1536,vacation,"hoping, childhood vacation, own kids, griswold..."
3738,lost translation,"tokyo, washed-up movie star shooting, tv comme..."
3868,love,"renee, fairy tales, renee, ’ s life, darker ta..."
3264,inkwell,"inkwell, 16-year-old boy, martha, vineyard"
152,kung fu panda,"continuing, legendary adventures, po, differen..."
3291,pathology,"medical student, ted grey, milo ventimiglia, e..."
3796,committed suicide,"neal cassady, beat life, tire yard, happy life..."
2269,butterfly,"sociopathic kidnapper, desperate pair"
314,prisoner,lucrative secret process


In [ ]:
def list_to_sentence(column):
    return column.apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

def remove_special_characters(column):
    return column.apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', ' ', x) if isinstance(x, str) else x)

def decontraction(column):
    contraction_patterns = [
        (r"won\'t", "will not"),
        (r"won\'t've", "will not have"),
        (r"can\'t", "cannot"),
        (r"can\'t've", "cannot have"),
        (r"n\'t", "not"),
        (r"n\'t've", "not have"),
        (r"\'re", "are"),
        (r"\'s", "is"),
        (r"\'d", "would"),
        (r"\'d've", "would have"),
        (r"\'ll", "will"),
        (r"\'ll've", "will have"),
        (r"\'t", "not"),
        (r"\'ve", "have"),
        (r"\'m", "am"),
        (r"shan\'t", "shall not"),
        (r"sha\n't", "shall not"),
        (r"o\'clock", "of the clock"),
        (r"y\'all", "you all"),
        (r"let\'s", "let us"),
        (r"ma\'am", "madam"),
        (r"they\'re", "they are"),
    ]
    for pattern in contraction_patterns:
        column = column.apply(lambda x: re.sub(pattern[0], pattern[1], x) if isinstance(x, str) else x)

    return column

def preprocess_text(column):
    column = column.apply(lambda x: re.sub(r'\d+|\W+', ' ', x) if isinstance(x, str) else x)
    column = column.apply(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)
    return column

def convert_to_lowercase(column):
    return column.apply(lambda x: x.lower() if isinstance(x, str) else x)

def remove_stopwords(column):
    stop_words = set(stopwords.words('english'))
    return column.apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]) if isinstance(x, str) else x)

def lemmatize_text(column):
    lemmatizer = WordNetLemmatizer()

    def get_wordnet_pos(tag):
        if tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('R'):
            return wordnet.ADV
        elif tag.startswith('J'):
            return wordnet.ADJ
        else:
            return wordnet.NOUN

    return column.apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos=get_wordnet_pos(tag)) for word, tag in pos_tag(x.split())]) if isinstance(x, str) else x)

def tokenize_text(column):
    return column.apply(lambda x: ' '.join(word_tokenize(x)) if isinstance(x, str) else x)

def drop_nan_rows(data):
    return data.dropna()

def preprocess_data(data):
    if isinstance(data, pd.DataFrame):
        cleaned_data = data.copy()
        cleaned_data.drop_duplicates(inplace=True)
        for col in cleaned_data.columns:
            cleaned_data[col] = list_to_sentence(cleaned_data[col])
            cleaned_data[col] = remove_special_characters(cleaned_data[col])
            cleaned_data[col] = decontraction(cleaned_data[col])
            cleaned_data[col] = preprocess_text(cleaned_data[col])
            cleaned_data[col] = convert_to_lowercase(cleaned_data[col])
            cleaned_data[col] = remove_stopwords(cleaned_data[col])
            cleaned_data[col] = lemmatize_text(cleaned_data[col])
            cleaned_data[col] = tokenize_text(cleaned_data[col])
        return cleaned_data

In [ ]:
'''quotes = preprocess_data(rquotes_data)
quotes.sample(10)'''

,Target,Clues
5167,art,love deep far
27839,full experience,humanity imperfection life
45071,,debasish mridha debasish mridha difference peo...
22362,old technique work world brutal mystery,love
14532,particular path,book literature self discovery
11469,n cry,lie life relationship
1365,black art black history black culture,history culture book
15909,love late death early seriously,bizarre death early funny humor late love time...
10956,body,funny life time time truth wisdom
15822,quick way wrong person,break relationship


In [ ]:
movies = preprocess_data(rmovies_data)
movies.sample(10)

,Target,Clues
4660,shelter,shelter important animal issue film uncovers p...
4269,ballad gregorio cortez,problem evolves real perpetrator male horse er...
3101,swindle,betty victor scam artists betty maurice multin...
3575,ii,dante randall shop send new horizon mooby disn...
3163,detention,cinderhella student body high school grizzly c...
4708,hero dirt,passionate bmx dirt jumper phin cooper attain ...
3000,insidious,new tale teenage girl lambert
3307,drive miss daisy,old jewish widow daisy werthan hoke initial me...
2718,clueless,shallow cher beverly high school see cher embo...
4561,bear fly elizabeth streb gravity,bear fly elizabeth streb human flight


In [ ]:
'''proverbs = preprocess_data(rproverbs_data)
proverbs.sample(10)'''

,Target,Clues
37711,knowledge right knowledge let man godlike task,knowledge
23842,forgotten,anna wintour fashion life quote quote time tim...
36718,may mine bad luck,bad luck fail failure hope love lover pain poe...
41490,creative great art,great pain creative
42533,particular level,islam philosophy
19805,together bobs well bob ross happy tree,art artist bob bob ross death humor kabob pain...
41767,phantasie ist wichtiger al wissen wissen ist b...,inspiration
12469,deep disappointment,justice love segregation
26735,small percentage,appreciation inspirational life live motivatio...
28337,speakin street distant shore,explore fantasy little gidding mystery poetry ...


In [ ]:
'''idioms = preprocess_data(ridioms_data)
idioms.sample(10)'''

,Target,Clues
159,pron attention,awareness employee complaint military personne...
70,new arrival,
175,pron leg,hollywood right sister leg fig trick someone l...
307,pron mouth,literally mouth figuratively upset protester p...
295,pron fish,
102,pron place pron sun,favorable position situation place secondary p...
120,pron closet,non heterosexual person sexuality secret month...
242,pron touch,college roommate possessive pronoun league eli...
542,calendar year,january december particular year tax purpose m...
90,pron roof,high level popular show house price inf roof f...


In [ ]:
words = preprocess_data(word_data)
words.sample(10)

,Target,Clues
106,pot,tea coffee transport glass
261,oranjestad aruba,america
425,san juan puerto rico,america
44,othello,william shakespeare
2,absalom absalom,william faulkner
82,poem leopardi,giacomo leopardi
38,cut board,knife food wood
56,costa rica,americas central america san jos
355,dublin ireland,europe
74,taxi driver,martin scorsese drama cab new york city


In [ ]:
news = preprocess_data(rnews_data)
news.sample(10)

,Target,Clues
2617,stosur,new woman champion french open roland garros
1506,sharapova,russia open single feed round tie spain
1314,murdoch,rupert murdoch news charlotte family
2366,bundesliga,borussia dortmund bundesliga point record fina...
3480,borisov,bate borisov german giant bayern munich champi...
1189,milan,carlos tevez future san siro ac milan internaz...
391,hoey,michael hoey last round charge u open champion...
2134,federer,roger federer grand slam jimmy connors major c...
1769,federer,roger federer ivan ljubicic croat
2777,sharapova,tennis star maria sharapova russia flag openin...


In [ ]:
dictionary = preprocess_data(rdictionary_data)
dictionary.sample(10)

,Target,Clues
10190,inst,present month
39934,freight,common carrier
38697,meltdown,march nuclear meltdown
27060,strengthen,gain strength
8759,musical,musical comedy characteristic
27528,bruise,wooden spoon small piece food preparation
42125,grave,pen carve
6861,light,light load physical weight
25510,imagination,popular imagination mental image
26931,work,right consistency


In [ ]:
'''phrases = preprocess_data(rphrase_data)
phrases.sample(10)'''

,Target,Clues
289,new family,david fenton new family david fenton
1400,previous host old platform,furthermore different device gain access previ...
626,home network residential web system,home network march residential web system march
1440,system binary structure,political situation hong kong binary structure...
1427,case single,h ei entire length h ei entire length
1032,current body,silver slipper silver slipper current body
455,internal process house procedure,individual intrapersonal communication interna...
1505,equal position identical status,equal position eighteenth hole identical statu...
438,character shy quality,miss italia contest miss italia contest shy qu...
1048,current body physical frame,current body noah physical frame noah


In [ ]:
data = pd.concat([words, dictionary, news, movies], ignore_index=True)
new_path = 'final_data.csv'
data.to_csv(new_path, index=False)
data.sample(10)

,Target,Clues
32965,loot,large collection priceless coin bank vault lib...
36296,wild thing,kelly van ryan richards bad girl suzie toller ...
12303,despite,legendary lack intense dislike
28151,traditionally,hanukah traditional manner
19223,stoop,path sag
5625,infinite,infinite ingenuity
10569,acromegalic,acromegalic jaw
14068,enigmaticness,jesus magnetism contradictory character
30977,signal,menu communicate
27639,improvidently,improvident manner
